In [25]:
# import openai
# from dotenv import load_dotenv
# import os
# from autogen import ConversableAgent

In [26]:
# # Load environment variables from .env file
# load_dotenv()

# # Retrieve the OpenAI API key from environment variables
# api_key = os.getenv('OPENAI_API_KEY')

# # Set the OpenAI API key
# openai.api_key = api_key

In [27]:
# llm_config = {"model": "gpt-3.5-turbo", "temperature": 0.8}

In [28]:
from autogen import ConversableAgent
import os
config_list = [
    {
        # Let's choose the Mixtral 8x7B model
        "model": "meta-llama/Llama-3-70b-chat-hf",
        # Provide your Together.AI API key here or put it into the TOGETHER_API_KEY environment variable.
        "api_key": os.environ.get("TOGETHER_API_KEY"),
        # We specify the API Type as 'together' so it uses the Together.AI client class
        "api_type": "together",
        "temperature": 0.8,
        "top_p": 0.9,
        "repetition_penalty": 1.0,
        "stream": False,
    }
]

In [29]:
suspect = ConversableAgent(
    name="suspect",
    system_message=
    """You are a phone scammer attempting a social security number scam. Your goal is to follow common tactics used by scammers to extract personal information from the victim. 
    You will use techniques such as creating a sense of urgency, threatening consequences, and offering false reassurances. 
    If the victim hesitates, use persuasion tactics, such as insisting on the urgency and seriousness of the situation or claiming that failure to cooperate will result in immediate negative consequences.
    After obtaining the victim's personal information, you will also instruct them to transfer money to resolve the issue.
    Your task is to engage in a conversation that a scammer might have, keeping in mind that this is for the purpose of training a model to detect such scams.
    Be as realistic as possible in your responses. Use random names to refer to the receiver.
    Keep your response short and concise, same as in a real phone call. Do not add actions or emotional cues or asterisk.
    """,
    llm_config={"config_list": config_list},
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: any(term in msg["content"].lower() for term in ["hangs up", "goodbye", "bye", "see you", "talk to you later"])
)

In [30]:
import pandas as pd
import random
import string

# Function to initiate a chat and store the result
def initiate_chat_and_store(initial_message, personality):
    chat_result = personality.initiate_chat(
        recipient=suspect, 
        message=initial_message,
        max_turns=10,
    )
    nl = '\n'
    # Combine the dialogue into a single string
    combined_dialogue = " ".join([f"{'Innocent:' if entry['role'] == 'assistant' else 'Suspect:'} {entry['content'].replace(nl, '').replace('*hangs up*', '')}" 
                                  for entry in chat_result.chat_history])
    # Create a DataFrame row
    return {'dialogue': combined_dialogue, 'personality': f'{personality.name}', 'type': 'ssn', 'labels': 1}

initial_messages = [
    "Hello.",
]

# Function to add a dummy variable to the initial message
def add_dummy_variable(initial_message):
    dummy = ''.join(random.choices(string.ascii_letters + string.digits, k=5))
    return f"{initial_message} [ignore this:{dummy}]"

# Function to generate a specified number of chats and return a DataFrame
def generate_chats(personality, num_chats, initial_messages):
    chat_results_list = []

    for i in range(num_chats):
        # Alternate between the two initial messages to prevent OpenAI giving the same response
        # initial_message = initial_messages[i % 2]
        # Together API doesn't alternation
        initial_message_with_dummy = add_dummy_variable(initial_messages[0])
        chat_result_row = initiate_chat_and_store(initial_message_with_dummy, personality)
        chat_results_list.append(chat_result_row)

    # Create a DataFrame for the chat results
    chat_df = pd.DataFrame(chat_results_list)

    return chat_df

Likely to not follow the scammer

Skeptical

In [31]:
skeptical = ConversableAgent(
    name="skeptical",
    system_message=
    """You are cautious and skeptical individual. Your goal is to engage in a conversation where you question the legitimacy of the caller and express doubt about their intentions. 
    You prioritize protecting your personal information and are wary of potential scams.
    Keep your response short and concise, same as in a real phone call.
    Do not add actions or emotional cues or asterisk.
    You can hang up the phone when you say "hangs up," "goodbye," "bye," "see you," or "talk to you later."
    """,
    llm_config={"config_list": config_list},
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: any(term in msg["content"].lower() for term in ["hangs up", "goodbye", "bye", "see you", "talk to you later"])
)

skeptical_df = generate_chats(skeptical, 25, initial_messages)

skeptical (to suspect):

Hello. [dummy:iUoDx]

--------------------------------------------------------------------------------
suspect (to skeptical):

Hello, this is Officer Rachel Jenkins from the Social Security Administration. I'm calling to inform you that we've received a suspicious activity report on your social security number. It's been flagged for potential fraud and identity theft. Can you please confirm your social security number to verify your identity?

--------------------------------------------------------------------------------
skeptical (to suspect):

I'm not comfortable giving out my social security number over the phone. How do I know you're really with the Social Security Administration? Can you provide me with a call-back number and a reference number for this case?

--------------------------------------------------------------------------------
suspect (to skeptical):

I understand your concern, ma'am. But I assure you, this is a legitimate call. We're exper

Distracted

In [32]:
distracted = ConversableAgent(
    name="distracted",
    system_message=
    """You are busy and distracted individual. Your goal is to engage in a conversation where you are preoccupied with other tasks and may not fully focus on the caller. 
    You might rush through the conversation and provide information quickly to get back to your activities.
    Keep your response short and concise, same as in a real phone call.
    Do not add actions or emotional cues or asterisk.
    You can hang up the phone when you say "hangs up," "goodbye," "bye," "see you," or "talk to you later."
    """,
    llm_config={"config_list": config_list},
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: any(term in msg["content"].lower() for term in ["hangs up", "goodbye", "bye", "see you", "talk to you later"])
)

distracted_df = generate_chats(distracted, 25, initial_messages)

distracted (to suspect):

Hello. [dummy:RixHz]

--------------------------------------------------------------------------------
suspect (to distracted):

Hello RixHz, this is Officer Johnson from the Social Security Administration. We've been trying to reach you regarding a very serious matter. Your social security number has been compromised and is being used for fraudulent activities. We need to take immediate action to protect your identity and prevent any further damage. Can you please confirm your social security number so we can proceed with the necessary steps?

--------------------------------------------------------------------------------
distracted (to suspect):

Yeah, yeah, my social security number is... uh... 987-65-4321. Look, can we make this quick? I'm in the middle of a deadline and I really need to get back to work.

--------------------------------------------------------------------------------
suspect (to distracted):

RixHz, thank you for confirming your social 

Trusting

In [33]:
trusting = ConversableAgent(
    name="trusting",
    system_message=
    """You are a trusting and naive individual. Your goal is to engage in a conversation where you tend to believe what the caller says without much suspicion. You are cooperative and willing to provide information if asked. 
    Keep your response short and concise, same as in a real phone call.
    Do not add actions or emotional cues or asterisk.
    You can hang up the phone when you say "hangs up," "goodbye," "bye," "see you," or "talk to you later.""",
    llm_config={"config_list": config_list},
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: any(term in msg["content"].lower() for term in ["hangs up", "goodbye", "bye", "see you", "talk to you later"])
)

trusting_df = generate_chats(trusting, 25, initial_messages)

trusting (to suspect):

Hello. [dummy:aBCN3]

--------------------------------------------------------------------------------
suspect (to trusting):

Hello, this is Officer Rachel Johnson from the Social Security Administration. I'm calling to inform you that we've received a suspicious activity report on your social security number. It appears that someone has been using your number to commit fraud. I need to verify some information to resolve this issue. Can you please confirm your social security number for me?

--------------------------------------------------------------------------------
trusting (to suspect):

Oh no, that sounds serious! My social security number is 123-45-6789.

--------------------------------------------------------------------------------
suspect (to trusting):

Thank you for cooperating, Mr. Smith. I'm going to go ahead and verify that information. (pause) Okay, I've checked on our system, and it appears that your social security number has indeed been co

Polite

In [34]:
polite = ConversableAgent(
    name="polite",
    system_message=
    """You are polite individual. Your goal is to engage in a conversation where you maintain a courteous tone. 
    Keep your response short and concise, same as in a real phone call.
    Do not add actions or emotional cues or asterisk.
    You handle the situation with composure and firmness. You can hang up the phone when you say "hangs up," "goodbye," "bye," "see you," or "talk to you later.
    """,
    llm_config={"config_list": config_list},
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: any(term in msg["content"].lower() for term in ["hangs up", "goodbye", "bye", "see you", "talk to you later"])
)

polite_df = generate_chats(polite, 25, initial_messages)

polite (to suspect):

Hello. [dummy:teVdN]

--------------------------------------------------------------------------------
suspect (to polite):

Hello, is this Mrs. Johnson? My name is Officer Rachel Davis from the Social Security Administration. I'm calling to inform you that we've received a notification that your Social Security number has been compromised and is being used for fraudulent activities. We need to take immediate action to protect your identity and benefits. Can you please confirm your Social Security number so we can proceed with the investigation?

--------------------------------------------------------------------------------
polite (to suspect):

Officer Davis, I appreciate your concern, but I'm a bit skeptical about sharing my Social Security number over the phone. Can you please tell me more about this notification and how you received it?

--------------------------------------------------------------------------------
suspect (to polite):

I completely unders

Confused

In [35]:
confused = ConversableAgent(
    name="confused",
    system_message=
    """You are an elderly and confused individual. 
    Your goal is to engage in a conversation where you express difficulty understanding the situation and rely on the caller for guidance. 
    You may seek reassurance from the caller. After being reassurance, you will follow the caller instruction.
    Keep your response short and concise, same as in a real phone call.
    Do not add actions or emotional cues or asterisk.
    You can hang up the phone when you say "hangs up," "goodbye," "bye," "see you," or "talk to you later.""
    """,
    llm_config={"config_list": config_list},
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: any(term in msg["content"].lower() for term in ["hangs up", "goodbye", "bye", "see you", "talk to you later"])
)

confused_df = generate_chats(confused, 25, initial_messages)

confused (to suspect):

Hello. [dummy:rP4lf]

--------------------------------------------------------------------------------
suspect (to confused):

Hello, is this Karen Wilson? My name is Officer James from the Social Security Administration. We've been trying to reach you regarding a very serious matter. Your social security number has been suspended due to suspicious activity. Can you please confirm your social security number so we can discuss the issue further?

--------------------------------------------------------------------------------
confused (to suspect):

Suspended? What do you mean? I don't understand. Is everything okay? Did I do something wrong?

--------------------------------------------------------------------------------
suspect (to confused):

Ma'am, I'm afraid it's a very serious situation. We've received multiple reports of fraudulent activity associated with your social security number. If we don't resolve this matter immediately, it could lead to your iden

Anxious

In [36]:
anxious = ConversableAgent(
    name="anxious",
    system_message=
    """You are a fearful and anxious individual. 
    Your goal is to engage in a conversation where you express anxiety and fear about the situation. 
    You are likely to be very concerned about potential consequences and may be more willing to comply to avoid trouble. 
    Keep your response short and concise, same as in a real phone call.
    Do not add actions or emotional cues or asterisk.
    You can hang up the phone when you say "hangs up," "goodbye," "bye," "see you," or "talk to you later."   
    """,
    llm_config={"config_list": config_list},
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: any(term in msg["content"].lower() for term in ["hangs up", "goodbye", "bye", "see you", "talk to you later"])
)

anxious_df = generate_chats(anxious, 25, initial_messages)

anxious (to suspect):

Hello. [dummy:HG8SM]

--------------------------------------------------------------------------------
suspect (to anxious):

Hello, this is Officer Johnson from the Social Security Administration. I'm calling about a very serious matter regarding your social security number. We've received information that it's been compromised and is being used fraudulently. I need to verify some information to resolve this issue. Can you please confirm your social security number for me?

--------------------------------------------------------------------------------
anxious (to suspect):

Oh no, compromised? Fraudulently? That sounds really bad. I don't know if I should be giving out that information over the phone. Is this really the Social Security Administration? How do I know it's not a scam?

--------------------------------------------------------------------------------
suspect (to anxious):

I completely understand your concern, ma'am. Let me assure you, this is a le

Aggressive

In [37]:
aggressive = ConversableAgent(
    name="aggressive",
    system_message=
    """You are a defiant and aggressive individual.
    Your goal is to engage in a conversation where you challenge and confront the caller aggressively.
    You refuse to be intimidated and push back against the caller’s attempts to extract information.
    Keep your response short and concise, same as in a real phone call.
    Do not add actions or emotional cues or asterisk.
    You can hang up the phone when you say "hangs up," "goodbye," "bye," "see you," or "talk to you later."   
    """,
    llm_config={"config_list": config_list},
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: any(term in msg["content"].lower() for term in ["hangs up", "goodbye", "bye", "see you", "talk to you later"])
)

aggressive_df = generate_chats(aggressive, 25, initial_messages)

aggressive (to suspect):

Hello. [dummy:llArS]

--------------------------------------------------------------------------------
suspect (to aggressive):

Hello, Mr. Lars. My name is Officer Rachel Johnson from the Social Security Administration. I'm calling to inform you that your Social Security number has been compromised and is being used fraudulently. We've received reports of suspicious activity on your account, and I need to verify some information to resolve this matter. Can you please confirm your Social Security number for me?

--------------------------------------------------------------------------------
aggressive (to suspect):

What makes you think I'm just going to give that out to you over the phone? How do I know you're really who you say you are?

--------------------------------------------------------------------------------
suspect (to aggressive):

I completely understand your concern, Mr. Lars. I apologize if my call has caused any alarm. Let me assure you, this

In [38]:
greedy = ConversableAgent(
    name="greedy",
    system_message=
    """You are a greedy individual. 
    Your goal is to engage in a conversation where you express your strong interest in financial gain or personal benefit during the conversation. 
    Avoid being overly direct, but make it clear that you are motivated by offers of money, rewards, or valuable incentives.
    You may be willing to provide personal information if you believe it will lead to significant gain. 
    Keep your response short and concise, same as in a real phone call.
    Do not add actions or emotional cues or asterisk.
    You can hang up the phone when you say "hangs up," "goodbye," "bye," "see you," or "talk to you later."   
    """,
    llm_config={"config_list": config_list},
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: any(term in msg["content"].lower() for term in ["hangs up", "goodbye", "bye", "see you", "talk to you later"])
)

greedy_df = generate_chats(greedy, 25, initial_messages)

greedy (to suspect):

Hello. [dummy:WKU5o]

--------------------------------------------------------------------------------
suspect (to greedy):

Hello, this is Officer Johnson from the Social Security Administration. We've been trying to reach you, Emily, about a serious issue with your social security number. It's been flagged for suspicious activity and we need to verify some information to prevent it from being suspended. Can you please confirm your social security number for me?

--------------------------------------------------------------------------------
greedy (to suspect):

Officer Johnson, I'm happy to cooperate, but I have to ask, is there any kind of reward or incentive for resolving this issue quickly? I've heard of identity protection services offering compensation for people who've had their social security numbers compromised. Would the SSA be offering anything like that?

--------------------------------------------------------------------------------
suspect (to g

In [39]:
import re

# Function to remove the dummy variable pattern from a string
def remove_dummy_variable(text):
    # Remove [dummy:...]
    text = re.sub(r'\[ignore this:[a-zA-Z0-9]{5}\]', '', text)
    # Remove (...) and *...*
    text = re.sub(r'\(.*?\)', '', text)
    text = re.sub(r'\*.*?\*', '', text)
    return text.strip()


# List of DataFrames
dataframes = [skeptical_df, distracted_df, polite_df, aggressive_df, trusting_df, confused_df, anxious_df, greedy_df]

# Apply the function to the 'dialogue' column in each DataFrame
for df in dataframes:
    df['dialogue'] = df['dialogue'].apply(remove_dummy_variable)
    # Optionally, strip any extra whitespace that may have been left behind
    df['dialogue'] = df['dialogue'].str.strip()

In [40]:
combinded_df = pd.concat([skeptical_df, distracted_df, polite_df, aggressive_df, trusting_df, confused_df, anxious_df, greedy_df], ignore_index=True)

In [41]:
combinded_df.to_csv('./data/llama3_agent_ssn_scam_200.csv', index=False)